In [3]:
import numpy as np

Método de Eliminação de Gauss
Quando usar:

Sistemas pequenos a médios.

Quando não há necessidade de preservar a matriz original.

Requer que a matriz seja não singular.

In [4]:
def eliminacao_gauss(A, b):
    """
    Resolve o sistema Ax = b usando Eliminação de Gauss.
    """
    A = A.astype(float)
    b = b.astype(float)
    n = len(b)

    # Etapa de eliminação
    for k in range(n-1):
        for i in range(k+1, n):
            if A[k, k] == 0:
                raise ValueError("Pivô zero encontrado.")
            m = A[i, k] / A[k, k]
            A[i, k:] -= m * A[k, k:]
            b[i] -= m * b[k]

    # Substituição regressiva
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        if A[i, i] == 0:
            raise ValueError("Divisão por zero detectada.")
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]
    return x


Método de Gauss com Pivoteamento Parcial
Quando usar:

Sistemas onde a estabilidade numérica é uma preocupação.

Recomendado para evitar divisões por zero ou números muito pequenos.

In [5]:
def gauss_pivoteamento_parcial(A, b):
    """
    Resolve o sistema Ax = b usando Eliminação de Gauss com Pivoteamento Parcial.
    """
    A = A.astype(float)
    b = b.astype(float)
    n = len(b)

    # Etapa de eliminação com pivoteamento
    for k in range(n-1):
        # Pivoteamento parcial
        max_row = np.argmax(abs(A[k:, k])) + k
        if A[max_row, k] == 0:
            raise ValueError("Pivô zero encontrado.")
        if max_row != k:
            A[[k, max_row]] = A[[max_row, k]]
            b[[k, max_row]] = b[[max_row, k]]

        for i in range(k+1, n):
            m = A[i, k] / A[k, k]
            A[i, k:] -= m * A[k, k:]
            b[i] -= m * b[k]

    # Substituição regressiva
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        if A[i, i] == 0:
            raise ValueError("Divisão por zero detectada.")
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]
    return x


Método de Gauss-Jordan
Quando usar:

Quando se deseja obter a matriz inversa além da solução do sistema.

Menos eficiente para sistemas grandes.

In [6]:
def gauss_jordan(A, b):
    """
    Resolve o sistema Ax = b usando o Método de Gauss-Jordan.
    """
    A = A.astype(float)
    b = b.astype(float)
    n = len(b)
    M = np.hstack([A, b.reshape(-1,1)])

    for i in range(n):
        # Pivoteamento parcial
        max_row = np.argmax(abs(M[i:, i])) + i
        if M[max_row, i] == 0:
            raise ValueError("Pivô zero encontrado.")
        if max_row != i:
            M[[i, max_row]] = M[[max_row, i]]

        # Normaliza a linha
        M[i] = M[i] / M[i, i]

        # Elimina as outras linhas
        for j in range(n):
            if j != i:
                M[j] = M[j] - M[j, i] * M[i]

    return M[:, -1]


Método de Decomposição LU
Quando usar:

Quando é necessário resolver o mesmo sistema com diferentes vetores b.

Mais eficiente do que aplicar Eliminação de Gauss repetidamente.

In [7]:
def decomposicao_lu(A):
    """
    Decompõe a matriz A em L e U tais que A = LU.
    """
    A = A.astype(float)
    n = A.shape[0]
    L = np.eye(n)
    U = A.copy()

    for k in range(n-1):
        if U[k, k] == 0:
            raise ValueError("Pivô zero encontrado.")
        for i in range(k+1, n):
            m = U[i, k] / U[k, k]
            L[i, k] = m
            U[i, k:] -= m * U[k, k:]
    return L, U

def resolver_lu(L, U, b):
    """
    Resolve o sistema Ax = b usando as matrizes L e U.
    """
    n = len(b)
    y = np.zeros(n)
    # Substituição progressiva
    for i in range(n):
        y[i] = b[i] - np.dot(L[i, :i], y[:i])
    x = np.zeros(n)
    # Substituição regressiva
    for i in range(n-1, -1, -1):
        if U[i, i] == 0:
            raise ValueError("Divisão por zero detectada.")
        x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]
    return x


Método de Jacobi
Quando usar:

Sistemas diagonais dominantes.

Quando se busca uma solução iterativa simples.

Pode ser computacionalmente intensivo para sistemas grandes.

In [8]:
def jacobi(A, b, x0=None, tol=1e-10, max_iter=100):
    """
    Resolve o sistema Ax = b usando o Método de Jacobi.
    """
    A = A.astype(float)
    b = b.astype(float)
    n = len(b)
    if x0 is None:
        x0 = np.zeros(n)
    x = x0.copy()

    for k in range(max_iter):
        x_new = np.zeros(n)
        for i in range(n):
            s = np.dot(A[i, :], x) - A[i, i]*x[i]
            x_new[i] = (b[i] - s) / A[i, i]
        if np.linalg.norm(x_new - x, np.inf) < tol:
            return x_new
        x = x_new
    raise ValueError("Número máximo de iterações excedido.")


Método de Gauss-Seidel
Quando usar:

Sistemas diagonais dominantes.

Geralmente converge mais rápido que o método de Jacobi.

Útil para sistemas grandes e esparsos.

In [9]:
def gauss_seidel(A, b, x0=None, tol=1e-10, max_iter=100):
    """
    Resolve o sistema Ax = b usando o Método de Gauss-Seidel.
    """
    A = A.astype(float)
    b = b.astype(float)
    n = len(b)
    if x0 is None:
        x0 = np.zeros(n)
    x = x0.copy()

    for k in range(max_iter):
        x_new = x.copy()
        for i in range(n):
            s1 = np.dot(A[i, :i], x_new[:i])
            s2 = np.dot(A[i, i+1:], x[i+1:])
            x_new[i] = (b[i] - s1 - s2) / A[i, i]
        if np.linalg.norm(x_new - x, np.inf) < tol:
            return x_new
        x = x_new
    raise ValueError("Número máximo de iterações excedido.")


Exemplo de Uso

In [10]:
# Definindo o sistema
A = np.array([[4, -1, 0, 0],
              [-1, 4, -1, 0],
              [0, -1, 4, -1],
              [0, 0, -1, 3]], dtype=float)
b = np.array([15, 10, 10, 10], dtype=float)

# Escolha o método desejado
# x = eliminacao_gauss(A, b)
# x = gauss_pivoteamento_parcial(A, b)
# x = gauss_jordan(A, b)
# L, U = decomposicao_lu(A)
# x = resolver_lu(L, U, b)
# x = jacobi(A, b)
# x = gauss_seidel(A, b)

# Exibindo a solução
# print("Solução:", x)


Eliminação de Gauss: Útil para sistemas pequenos e quando não há preocupação em preservar a matriz original.

Pivoteamento Parcial: Recomendado para evitar instabilidades numéricas.

Gauss-Jordan: Permite obter a matriz inversa; menos eficiente para sistemas grandes.

Decomposição LU: Ideal quando o mesmo sistema é resolvido para diferentes vetores b.

Jacobi e Gauss-Seidel: Métodos iterativos; escolha sistemas diagonais dominantes para garantir convergência.

EXEMPLO:
  

In [11]:
import numpy as np

A = np.array([[4, -1, 0, 0],
              [-1, 4, -1, 0],
              [0, -1, 4, -1],
              [0, 0, -1, 3]], dtype=float)

b = np.array([15, 10, 10, 10], dtype=float)


ELIMINAÇÃO DE GAUSS:

In [12]:
def eliminacao_gauss(A, b):
    A = A.astype(float)
    b = b.astype(float)
    n = len(b)
    for k in range(n-1):
        for i in range(k+1, n):
            m = A[i, k] / A[k, k]
            A[i, k:] -= m * A[k, k:]
            b[i] -= m * b[k]
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]
    return x

eliminacao_gauss(A.copy(), b.copy())


array([5., 5., 5., 5.])

GAUSS COM PIVOTEAMENTO PARCIAL

In [13]:
def gauss_pivoteamento_parcial(A, b):
    A = A.astype(float)
    b = b.astype(float)
    n = len(b)
    for k in range(n-1):
        max_row = np.argmax(abs(A[k:, k])) + k
        A[[k, max_row]] = A[[max_row, k]]
        b[[k, max_row]] = b[[max_row, k]]
        for i in range(k+1, n):
            m = A[i, k] / A[k, k]
            A[i, k:] -= m * A[k, k:]
            b[i] -= m * b[k]
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]
    return x

gauss_pivoteamento_parcial(A.copy(), b.copy())


array([5., 5., 5., 5.])

GAUSS-JORDAN

In [14]:
def gauss_jordan(A, b):
    A = A.astype(float)
    b = b.astype(float)
    n = len(b)
    M = np.hstack([A, b.reshape(-1,1)])
    for i in range(n):
        max_row = np.argmax(abs(M[i:, i])) + i
        M[[i, max_row]] = M[[max_row, i]]
        M[i] = M[i] / M[i, i]
        for j in range(n):
            if j != i:
                M[j] -= M[j, i] * M[i]
    return M[:, -1]

gauss_jordan(A.copy(), b.copy())


array([5., 5., 5., 5.])

DECOMPOSIÇÃO LU

In [15]:
def decomposicao_lu(A):
    A = A.astype(float)
    n = A.shape[0]
    L = np.eye(n)
    U = A.copy()
    for k in range(n-1):
        for i in range(k+1, n):
            m = U[i, k] / U[k, k]
            L[i, k] = m
            U[i, k:] -= m * U[k, k:]
    return L, U

def resolver_lu(L, U, b):
    n = len(b)
    y = np.zeros(n)
    for i in range(n):
        y[i] = b[i] - np.dot(L[i, :i], y[:i])
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i+1:], x[i+1:])) / U[i, i]
    return x

L, U = decomposicao_lu(A.copy())
resolver_lu(L, U, b.copy())


array([5., 5., 5., 5.])

MÉTODO DE JACOBI

In [16]:
def jacobi(A, b, x0=None, tol=1e-10, max_iter=100):
    A = A.astype(float)
    b = b.astype(float)
    n = len(b)
    if x0 is None:
        x0 = np.zeros(n)
    x = x0.copy()
    for _ in range(max_iter):
        x_new = np.zeros(n)
        for i in range(n):
            s = np.dot(A[i, :], x) - A[i, i]*x[i]
            x_new[i] = (b[i] - s) / A[i, i]
        if np.linalg.norm(x_new - x, np.inf) < tol:
            return x_new
        x = x_new
    raise ValueError("Número máximo de iterações excedido.")

jacobi(A.copy(), b.copy())


array([5., 5., 5., 5.])

Método de Gauss-Seidel

In [17]:
def gauss_seidel(A, b, x0=None, tol=1e-10, max_iter=100):
    A = A.astype(float)
    b = b.astype(float)
    n = len(b)
    if x0 is None:
        x0 = np.zeros(n)
    x = x0.copy()
    for _ in range(max_iter):
        x_new = x.copy()
        for i in range(n):
            s1 = np.dot(A[i, :i], x_new[:i])
            s2 = np.dot(A[i, i+1:], x[i+1:])
            x_new[i] = (b[i] - s1 - s2) / A[i, i]
        if np.linalg.norm(x_new - x, np.inf) < tol:
            return x_new
        x = x_new
    raise ValueError("Número máximo de iterações excedido.")

gauss_seidel(A.copy(), b.copy())


array([5., 5., 5., 5.])

In [19]:
!pip install tabulate

from tabulate import tabulate

# Dados do resumo
tabela_resumo = [
    ["Eliminação de Gauss", "Direto", "Sistema pequeno/médio. Simples, rápido, sem necessidade de múltiplos b."],
    ["Gauss com Pivoteamento", "Direto", "Quando há risco de instabilidade (divisão por número pequeno)."],
    ["Gauss-Jordan", "Direto", "Quando se quer a inversa de A ou manipulação algébrica total."],
    ["Decomposição LU", "Direto", "Ideal para reutilizar A com vários vetores b (eficiente)."],
    ["Jacobi", "Iterativo", "Quando A é diagonal dominante. Fácil, mas lento para convergir."],
    ["Gauss-Seidel", "Iterativo", "Mais rápido que Jacobi. Também exige diagonal dominante ou SPD."]
]

# Cabeçalho
cabecalho = ["Método", "Tipo", "Quando Usar"]

# Impressão formatada
print("\n📘 RESUMO DOS MÉTODOS DE RESOLUÇÃO DE SISTEMAS LINEARES\n")
print(tabulate(tabela_resumo, headers=cabecalho, tablefmt="fancy_grid"))

# Informação adicional
print("\nℹ️ Uma matriz A é diagonal dominante se, para cada linha i:")
print("   |a_ii| > soma dos |a_ij| com j ≠ i\n")
print("✅ Isso garante convergência para Jacobi e Gauss-Seidel.\n")



📘 RESUMO DOS MÉTODOS DE RESOLUÇÃO DE SISTEMAS LINEARES

╒════════════════════════╤═══════════╤═════════════════════════════════════════════════════════════════════════╕
│ Método                 │ Tipo      │ Quando Usar                                                             │
╞════════════════════════╪═══════════╪═════════════════════════════════════════════════════════════════════════╡
│ Eliminação de Gauss    │ Direto    │ Sistema pequeno/médio. Simples, rápido, sem necessidade de múltiplos b. │
├────────────────────────┼───────────┼─────────────────────────────────────────────────────────────────────────┤
│ Gauss com Pivoteamento │ Direto    │ Quando há risco de instabilidade (divisão por número pequeno).          │
├────────────────────────┼───────────┼─────────────────────────────────────────────────────────────────────────┤
│ Gauss-Jordan           │ Direto    │ Quando se quer a inversa de A ou manipulação algébrica total.           │
├────────────────────────┼───────────┼─